In [105]:
import pandas as pd
import os
import datetime
import numpy as np
import time
import Aemet_API as api
import requests

In [106]:
time.sleep(65)

In [107]:
estaciones = api.estaciones()
estaciones.columns = ['idema_code', 'meteorology_station', 'region']

In [108]:
df = pd.read_csv(os.path.join('..', 'Datos', 'Transformados', 'Limpios', 'df_limpio.csv'))

In [109]:
df['checkin_time'] = pd.to_datetime(df['checkin_time'])
df['checkout_time'] = pd.to_datetime(df['checkout_time'])

In [110]:
df['checkin_day_n'] = df['checkin_time'].dt.day.astype('str')
df['checkin_month_n'] = df['checkin_time'].dt.month.astype('str')
df['checkin_year_n'] = df['checkin_time'].dt.year.astype('str')
df['checkout_day_n'] = df['checkout_time'].dt.day.astype('str')
df['checkout_month_n'] = df['checkout_time'].dt.month.astype('str')
df['checkout_year_n'] = df['checkin_time'].dt.year.astype('str')

hoteles = df['asset'].unique().tolist()
idx_hotel = []
for i in df['asset']:
    idx_hotel.append(hoteles.index(i))
ciudades = ['DONOSTIA', 'VITORIA', 'BILBAO', 'BILBAO', 'VALENCIA', 'VALENCIA', 'MADRID', 'MALAGA', 'GRANADA', 'MALAGA', 'CORDOBA', 'PAMPLONA']
cities = []
for i in idx_hotel:
    cities.append(ciudades[i])
df['city'] = cities
df['city'] = df['city'].astype('category')
ciudad_provincia = {'DONOSTIA': 'GIPUZKOA', 'VITORIA': 'ARABA/ALAVA', 'BILBAO': 'BIZKAIA', 'VALENCIA': 'VALENCIA', 'MADRID': 'MADRID', 'MALAGA': 'MALAGA', 'GRANADA': 'GRANADA', 'CORDOBA': 'CORDOBA', 'PAMPLONA': 'NAVARRA'}
df['region'] = df['city'].map(ciudad_provincia)
df['idema_code'] = df.merge(right = estaciones, on = 'region', how = 'left')['idema_code']

In [111]:
time.sleep(60)

In [112]:
df_meteo = pd.DataFrame()
wait = 61
wait2 = 30
for idema_code in df['idema_code'].unique().tolist():
      idema = str(idema_code)
      df_meteo_to_join_1 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 1, day = 1), 
                                     fecha_fin = api.fechador(year = 2023, month = 4, day = 30),
                                     idema = idema,
                                     espera = wait2)
      if isinstance(df_meteo_to_join_1, int):
            while isinstance(df_meteo_to_join_1, int):
                  time.sleep(wait)
                  df_meteo_to_join_1 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 1, day = 1), 
                                                fecha_fin = api.fechador(year = 2023, month = 4, day = 30),
                                                idema = idema,
                                                espera = wait2)
      #################################################################################################
      df_meteo_to_join_2 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 5, day = 1),
                                     fecha_fin = api.fechador(year = 2023, month = 8, day = 31),
                                     idema = idema,
                                     espera = wait2)
      if isinstance(df_meteo_to_join_2, int):
            while isinstance(df_meteo_to_join_1, int):
                  time.sleep(wait)
                  df_meteo_to_join_2 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 5, day = 1), 
                                                fecha_fin = api.fechador(year = 2023, month = 8, day = 31),
                                                idema = idema,
                                                espera = wait2)
      #################################################################################################  
      df_meteo_to_join_3 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 9, day = 1),
                                     fecha_fin = api.fechador(year = 2023, month = 12, day = 31),
                                     idema = idema,
                                     espera = wait2)
      if isinstance(df_meteo_to_join_3, int):
            while isinstance(df_meteo_to_join_1, int):
                  time.sleep(wait)
                  df_meteo_to_join_3 = api.aemet(fecha_ini = api.fechador(year = 2023, month = 9, day = 1),
                                                fecha_fin = api.fechador(year = 2024, month = 1, day = 31),
                                                idema = idema,
                                                espera = wait2)
      #################################################################################################
      df_meteo_to_join = pd.concat([df_meteo_to_join_1, df_meteo_to_join_2, df_meteo_to_join_3], axis = 0)
      df_meteo = pd.concat([df_meteo, df_meteo_to_join], axis = 0)
      #################################################################################################
      del df_meteo_to_join
      del df_meteo_to_join_1
      del df_meteo_to_join_2
      del df_meteo_to_join_3

Request 429
Error: L�mite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
Request 429
Error: L�mite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
Request 429
Error: L�mite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
Request 429
Error: L�mite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
Request 429
Error: L�mite de peticiones o caudal por minuto excedido para este usuario. Espere al siguiente minuto.
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Request 200
Código de petición de la API: 200
Reques

In [113]:
df_meteo = df_meteo.reset_index()
if 'index' in df_meteo.columns.to_list():
    df_meteo = df_meteo[df_meteo.columns.to_list()[1:]]

In [122]:
df = df[df['lenght_of_stay'] < 31]

In [123]:
lista_fecha_checkin = []
lista_fecha_checkout = []
for i in range(df.shape[0]):
    f_checkin = api.fechador(year = df['checkin_year_n'].tolist()[i], 
                             month = df['checkin_month_n'].tolist()[i], 
                             day = df['checkin_day_n'].tolist()[i])
    f_checkout = api.fechador(year = df['checkout_year_n'].tolist()[i], 
                             month = df['checkout_month_n'].tolist()[i], 
                             day = df['checkout_day_n'].tolist()[i])
    lista_fecha_checkin.append(f_checkin[0:10])
    lista_fecha_checkout.append(f_checkout[0:10])
df_indices_meteo = pd.DataFrame({'F_C_I': lista_fecha_checkin, 'F_C_O': lista_fecha_checkout, 'IDEMA': df['idema_code']})

In [124]:
def estadisticos(var, df = df_meteo):
    import numpy as np
    import pandas as pd
    v = str(var)
    v_meteorologica = df[v].str.replace(',', '.').astype('float').to_list()
    meteo_min = min(v_meteorologica)
    meteo_Q1 = np.quantile(v_meteorologica, 0.25)
    meteo_Q2 = np.quantile(v_meteorologica, 0.5)
    meteo_Q3 = np.quantile(v_meteorologica, 0.75)
    meteo_med = np.mean(v_meteorologica)
    meteo_IQR = meteo_Q3 - meteo_Q1
    meteo_max = max(v_meteorologica)
    meteo_std = np.std(v_meteorologica)
    instancia_meteo_var = [float(meteo_min), 
                            float(meteo_Q1), 
                            float(meteo_Q2), 
                            float(meteo_Q3), 
                            float(meteo_max), 
                            float(meteo_IQR),
                            float(meteo_med), 
                            float(meteo_std)]
    instancia_var = ['MIN', 'Q1', 'Q2', 'Q3', 'MAX', 'IQR', 'MEAN', 'STD']
    meteo_df = np.transpose(pd.DataFrame({'VALOR': instancia_meteo_var}))
    meteo_df.columns = instancia_var
    return meteo_df

In [125]:
var_meteorologicas = ['tmin', 'tmax', 'tmed', 'prec', 'sol', 'velmedia', 'racha', 'hrMedia']

In [126]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
great_meteo_stat = pd.DataFrame()
for iter in range(df_indices_meteo.shape[0]):
    idema_codigo = df_indices_meteo['IDEMA'].to_list()[iter]
    lim_inf = df_indices_meteo['F_C_I'].tolist()[iter]
    lim_sup = df_indices_meteo['F_C_O'].tolist()[iter]
    boolean_serie_1 = (df_meteo[df_meteo['indicativo'] == str(idema_codigo)]['fecha'] == str(lim_inf))
    range_inf = boolean_serie_1.reset_index()[boolean_serie_1.to_list()]['index']
    boolean_serie_2 = (df_meteo[df_meteo['indicativo'] == str(idema_codigo)]['fecha'] == str(lim_sup))
    range_sup = boolean_serie_2.reset_index()[boolean_serie_2.to_list()]['index']
    meteo_range = range_inf.to_list() + range_sup.to_list()
    var1029 = df_meteo.iloc[meteo_range[0]:meteo_range[1]]
    if 'Ip' in var1029['prec'].to_list():
        var1029['prec'].iloc[var1029['prec'].to_list().index('Ip')] = '0,0'
    for ser in var1029.columns:
        serie = str(ser)
        var1029[serie] = var1029[serie].astype('str')
    meteo_stat_instance = pd.DataFrame()
    for var in var_meteorologicas:
        meteo_stat = estadisticos(str(var), df = var1029)
        meteo_stat_instance = pd.concat([meteo_stat_instance, meteo_stat], axis = 1)
    great_meteo_stat = pd.concat([great_meteo_stat, meteo_stat_instance], axis = 0)

ValueError: min() iterable argument is empty

In [ ]:
great_meteo_stat

In [ ]:
iter = 0

ini = df_meteo['fecha'].to_list().index(lista_fecha_checkin[iter])
fin = df_meteo['fecha'].to_list().index(lista_fecha_checkout[iter])

df_meteo.iloc[ini:fin]
estadisticos(var = 'tmin', df = df_meteo.iloc[ini:fin])

In [ ]:
# import time
# var_meteorologicas = ['tmin', 'tmax', 'tmed', 'prec']
# df_to_join = pd.DataFrame()
# for ind in range(df.shape[0]):
#     datos = api.aemet(fecha_ini = df['checkin_date_parsed'][ind], fecha_fin = df['checkout_date_parsed'][ind], idema = df['idema_code'][0])
#     instancia_meteo = []
#     meteo_len = datos.shape[0]
#     instancia_meteo.append(meteo_len)
#     for var in var_meteorologicas:
#         var_str = str(var)
#         v_meteorologica = datos[var_str].str.replace(',', '.').astype('float').to_list()    
#         meteo_min = min(v_meteorologica)
#         meteo_Q1 = np.quantile(v_meteorologica, 0.25)
#         meteo_Q2 = np.quantile(v_meteorologica, 0.5)
#         meteo_Q3 = np.quantile(v_meteorologica, 0.75)
#         meteo_med = np.mean(v_meteorologica)
#         meteo_IQR = meteo_Q3 - meteo_Q1
#         meteo_max = max(v_meteorologica)
#         meteo_std = np.std(v_meteorologica)
#         instancia_meteo_var = [float(meteo_min), 
#                                float(meteo_Q1), 
#                                float(meteo_Q2), 
#                                float(meteo_med), 
#                                float(meteo_Q3), 
#                                float(meteo_max), 
#                                float(meteo_IQR), 
#                                float(meteo_std)]
#         instancia_meteo += instancia_meteo_var
#     df_meteo = pd.DataFrame(instancia_meteo)
#     df_instancia_meteo = df_meteo.transpose()
#     df_instancia_meteo.columns = ['dias_estancia',
#                               'min_temp_min_ºC',
#                               'Q1_temp_min_ºC',
#                               'median_temp_min_ºC',
#                               'mean_temp_min_ºC',
#                               'Q3_temp_min_ºC',
#                               'max_temp_min_ºC',
#                               'IQR_temp_min_ºC',
#                               'desv_estandar_temp_min_ºC',
#                               'min_temp_max_ºC',
#                               'Q1_temp_max_ºC',
#                               'median_temp_max_ºC',
#                               'mean_temp_max_ºC',
#                               'Q3_temp_max_ºC',
#                               'max_temp_max_ºC',
#                               'IQR_temp_max_ºC',
#                               'desv_estandar_temp_max_ºC',
#                               'min_temp_media_ºC',
#                               'Q1_temp_media_ºC',
#                               'median_temp_media_ºC',
#                               'mean_temp_media_ºC',
#                               'Q3_temp_media_ºC',
#                               'max_temp_media_ºC',
#                               'IQR_temp_media_ºC',
#                               'desv_estandar_temp_media_ºC',
#                               'min_temp_precip_acum_mm',
#                               'Q1_temp_precip_acum_mm',
#                               'median_temp_precip_acum_mm',
#                               'mean_temp_precip_acum_mm',
#                               'Q3_temp_precip_acum_mm',
#                               'max_temp_precip_acum_mm',
#                               'IQR_temp_precip_acum_mm',
#                               'desv_estandar_temp_precip_acum_mm']
#     df_to_join = pd.concat([df_to_join, df_instancia_meteo], axis = 0)
#     time.sleep(1)